<div class="alert alert-block alert-success">
<b>Imported libraries</b>
</div>

In [1]:
import pandas as pd
import datetime as dt
import statsmodels.api as sm
from pandas_datareader import data

C:\Users\Sif\Anaconda3\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
#For a brazilian stock as FLRY3 you have to put a .SA so it will be FLRY3.SA
df = data.DataReader(['AAPL', 'AMZN'], 'yahoo', start = '2018/12/01', end = '2019/12/31')
df.head()

Attributes  Adj Close                   Close                    High  \
Symbols          AAPL         AMZN       AAPL         AMZN       AAPL   
Date                                                                    
2018-12-03  45.063950  1772.359985  46.205002  1772.359985  46.235001   
2018-12-04  43.081646  1668.400024  44.172501  1668.400024  45.597500   
2018-12-06  42.601311  1699.189941  43.680000  1699.189941  43.695000   
2018-12-07  41.082272  1629.130005  42.122501  1629.130005  43.622501   
2018-12-10  41.352917  1641.030029  42.400002  1641.030029  42.522499   

Attributes                     Low                    Open               \
Symbols            AMZN       AAPL         AMZN       AAPL         AMZN   
Date                                                                      
2018-12-03  1778.339966  45.302502  1730.000000  46.115002  1769.459961   
2018-12-04  1770.339966  44.067501  1665.000000  45.237499  1756.000000   
2018-12-06  1701.050049  42.605000  1609.849976  42.939999  1614.869995   
2018-12-07  1718.930054  42.075001  1625.459961  43.372501  1705.069946   
2018-12-10  1657.989990  40.832500  1590.869995  41.250000  1623.839966   

Attributes       Volume             
Symbols            AAPL       AMZN  
Date                                
2018-12-03  163210000.0  6862300.0  
2018-12-04  165377200.0  8694500.0  
2018-12-06  172393600.0  8789400.0  
2018-12-07  169126400.0  7576100.0  
2018-12-10  248104000.0  7494800.0

In [3]:
adj = df['Adj Close']
adj.head()

Symbols,AAPL,AMZN
Date,,
2018-12-03,45.063950,1772.359985
2018-12-04,43.081646,1668.400024
2018-12-06,42.601311,1699.189941
2018-12-07,41.082272,1629.130005
2018-12-10,41.352917,1641.030029


In [4]:
# 10D = 10 Days, M = 1 Month, Q = 1 Quarter....
# Don't use -> .asfreq('M', method = 'ffill') or .ffill()
monthly_price = adj.resample('M').last()
monthly_price.head()

Symbols,AAPL,AMZN
Date,,
2018-12-31,38.461140,1501.969971
2019-01-31,40.582428,1718.729980
2019-02-28,42.399570,1639.829956
2019-03-31,46.513416,1780.750000
2019-04-30,49.138439,1926.520020


In [5]:
# Percentage change of values by month
price_pct = monthly_price.pct_change()
# to remove Nan value use: .dropna()
price_pct

Symbols,AAPL,AMZN
Date,,
2018-12-31,NaN,NaN
2019-01-31,0.055154,0.144317
2019-02-28,0.044777,-0.045906
2019-03-31,0.097026,0.085936
2019-04-30,0.056436,0.081859
2019-05-31,-0.124213,-0.078613
2019-06-30,0.130519,0.066792
2019-07-31,0.076394,-0.014179
2019-08-31,-0.016461,-0.048474


In [6]:
# Cumulative Return
cumulative = (1+price_pct).cumprod()
# Could use .iloc[0] = 1 to start the first period with 1
cumulative.head()

Symbols,AAPL,AMZN
Date,,
2018-12-31,NaN,NaN
2019-01-31,1.055154,1.144317
2019-02-28,1.102400,1.091786
2019-03-31,1.209361,1.185610
2019-04-30,1.277613,1.282662


<div class="alert alert-block alert-info">
<b>Extra: Getting multiple stock's beta</b>
</div>

In [7]:
end = dt.datetime.now()
start = dt.date(end.year - 5, end.month - 1, end.day)
start_rf = dt.date(start.year, start.month + 1, start.day)

In [8]:
rm = data.DataReader(['^GSPC'], 'yahoo', start, end)['Adj Close']
rm = rm.resample('M').last().pct_change()
rm.head()

Symbols,^GSPC
Date,
2016-03-31,NaN
2016-04-30,0.002699
2016-05-31,0.015325
2016-06-30,0.000911
2016-07-31,0.035610


In [9]:
rf = data.DataReader('DGS1MO', 'fred', start_rf, end)
rf = rf.resample('M').last()
rf = rf/100
rf.head()

,DGS1MO
DATE,
2016-04-30,0.0016
2016-05-31,0.0027
2016-06-30,0.0020
2016-07-31,0.0019
2016-08-31,0.0026


In [10]:
df = pd.DataFrame({'rm': rm['^GSPC'], 
                   'rf': rf['DGS1MO']}).dropna()
df.head()

,rm,rf
Date,,
2016-04-30,0.002699,0.0016
2016-05-31,0.015325,0.0027
2016-06-30,0.000911,0.0020
2016-07-31,0.035610,0.0019
2016-08-31,-0.001219,0.0026


In [11]:
def beta(assets, start, end):
    frame = pd.DataFrame()
    frame = data.DataReader(assets, 'yahoo', start, end)['Adj Close']
    frame = frame.resample('M').last().pct_change().dropna()
    for i in frame.columns:
        X1 = sm.add_constant(df['rm'])
        beta = sm.OLS(endog = frame[i], exog = X1, missing = 'drop').fit().params[1]
        print(i,'Beta (5 Years - Monthly):', beta)

In [12]:
assets = ['GOOGL', 'TSLA', 'AAPL', 'MSFT','FB']
beta(assets, start, end)

GOOGL Beta (5 Years - Monthly): 1.0158819312812004
TSLA Beta (5 Years - Monthly): 1.9918644697983834
AAPL Beta (5 Years - Monthly): 1.2277435750129266
MSFT Beta (5 Years - Monthly): 0.8140688924882623
FB Beta (5 Years - Monthly): 1.268755511086035
